In [1]:
import sys
from pathlib import Path

# Add project root to Python path
sys.path.append('/app')

from src.extractors.youtube_extractor import YouTubeKidsExtractor

# Initialize extractor
extractor = YouTubeKidsExtractor()
extractor.ensure_data_directories()

# Test channel extraction
channel_id = "UCbCmjCuTUZos6Inko4u57UQ"  # Cocomelon
print("\nFetching channel details...")
channel_data = extractor.get_channel_details(channel_id)
if channel_data:
    extractor.save_data(channel_data, "channel_details")

# Test video extraction with date range
print("\nFetching videos from 2023...")
videos, quota_used = extractor.get_video_details(
    channel_id,
    start_date="2023-01-01T00:00:00",
    end_date="2023-01-31T23:59:59"
)

if videos:
    extractor.save_data(videos, "video_details")
    print(f"\nFetched {len(videos)} videos")
    print(f"API quota units used: {quota_used}")


Fetching channel details...
Data saved to data/raw/channel_details_20241223_235839.json

Fetching videos from 2023...
Data saved to data/raw/video_details_20241223_235839.json

Fetched 9 videos
API quota units used: 101
